# Trees

**`Binary trees`** are hierarchical data structures. They are drawn upside down, with the **`root node`** at the top, and the **`leaves`** at the bottom

Trees are similar to **`linked lists`** in the sense that the root holds references to its child nodes, but the difference is that nodes can have multiple children instead of just one. It follows these rules:

> Each node has a value and may have a list of childern
> 
> Children can have a single paren

Trees aren't useful if they are not ordered in some way. One of the most common types of ordered tree is a **`Binary Search Tree`** or **`BST`**. A BST has additional constraints:

1. Each node has at most two children
2. *Left child's* value must be less than its parent's value
3. *Right child's* value must be greater than its parent's value
4. No two nodes in the **`BST`** can have the same value

The building blocks of **`BST`** are **nodes**. Each node is technically also a full BST, with itself as the root node. In the examples of this chapter, each node will represent a user.

Each node has three properties:
* `value`. A **User** object, which additionally has a `name` and `ID` properties. The ID will be compared to place each node into the **`BST`**
* `left`. Left child of the node, which is another BSTNode or None
* `right`. Right child of the node, which is another BSTNode or None

## **`get_min` and `get_max`**

The following example implements the **`get_min`** and **`get_max`** methods to the **`BSTNode`** class. These algorithms are simple since the binary search tree is ordered. This causes that the leftmost value in the tree will always be the smallest value too, and the same with the rightmost being the larger value

In [1]:
#User class
import random


class User:
    def __init__(self, id):
        self.id = id
        user_names = [
            "Blake",
            "Ricky",
            "Shelley",
            "Dave",
            "George",
            "John",
            "James",
            "Mitch",
            "Williamson",
            "Burry",
            "Vennett",
            "Shipley",
            "Geller",
            "Rickert",
            "Carrell",
            "Baum",
            "Brownfield",
            "Lippmann",
            "Moses",
        ]
        self.user_name = f"{user_names[id % len(user_names)]}#{id}"

    def __eq__(self, other):
        return isinstance(other, User) and self.id == other.id

    def __lt__(self, other):
        return isinstance(other, User) and self.id < other.id

    def __gt__(self, other):
        return isinstance(other, User) and self.id > other.id

    def __repr__(self):
        return "".join(self.user_name)


def get_users(num):
    random.seed(1)
    users = []
    ids = []
    for i in range(num * 3):
        ids.append(i)
    random.shuffle(ids)
    ids = ids[:num]
    for id in ids:
        user = User(id)
        users.append(user)
    return users

In [ ]:
#BSTNode class
class BSTNode:
    def __init__(self, val=None):
        self.left = None
        self.right = None
        self.val = val
        
    #These are some of the simpler BST algorithms
    def get_min(self):
        if self.left is None:
            return self.val
        elif self.left is not None:
            return self.left.get_min()

    def get_max(self):
        if self.right is None:
            return self.val
        elif self.right is not None:
            return self.right.get_max()

## **`insert()`** method

Each node has the same properties: val, right and left, and the same methods that are defined inside the class, so to implement these methods, most of them use recursion to traverse the tree

In [ ]:
class BSTNode:
    def insert(self, val): #O(log(n))

        if self.val is None:
            self.val = val
            return
        
        if self.val == val:
             return

        #if the left and right does not exits 
        if val < self.val and self.left is None:
            self.left = BSTNode(val)

        elif val > self.val and self.right is None:
            self.right= BSTNode(val)

        #if the left and right exists, call recursively the method over the child
        elif val < self.val and self.left:
            self.left.insert(val)

        elif val > self.val and self.right:
            self.right.insert(val)

# **`delete()`** method

This is a trickier method, because if the node to be deleted has two children, you have to pick the leftmost value of the right child to conserve the **BST** property that says that the left child must be less than the current node, and the right child must be grater than the current node.  To make sure this happens, the algorithm should iterate over the right-node's left children to find the lower value. Then, this node (with the lower value) will replace the node to be deleted

In [4]:
class BSTNode:
    def delete(self, val): #O(log(n))
        #Empty tree or a leaf node where deletion has ocurred
        if self.val is None:
            return None
        
        if val < self.val:
            if self.left:
                self.left = self.left.delete(val)
                return self

        if val > self.val:
            if self.right is not None:
                self.right = self.right.delete(val)
                return self

        #value to delete found
        if val == self.val:
            #replacing the node to delete with its child if there is only one of them
            if not self.right:
                return self.left
            if not self.left:
                return self.right

            #Deleting a node with two children
            if self.right and self.left:
                succesor = self.right
                #This ensures that the succesor will be the smallest value from the right
                #so the rule of l < current node < r is actually followed
                while succesor.left is not None:
                    succesor = succesor.left
                self.val = succesor.val
                #This deletes the values on the right recursively, so the whole BST is 
                #restructured around the deleted val
                self.right = self.right.delete(succesor.val)
                return self

## Traversal order : **`preorder()`**, **`postorder()`**, **`inorder()`**

These functions use recursion to dive into each subtree of the current node. For example, for the **`inorder()`**:

1. The code goes as far left as possible, until it hits a node whose **`left`** is **`None`**
2. Then it appends the current node's value to the list
3. Then recurses on the **`self.right`** of the current node, if any
4. This continues up to the root, then down all the right branches in the same

Summary:

1. Recurse left if it exists
2. Append current value
3. Recurse right if it exists

Example:

For the following tree:

    > 7
    
        > 6
        
`> 4`

    > 2
    
        > 1
        

> **`preorder()`**. Current first, then left, then right

`[4, 2, 1, 7, 6]`

> **`postorder()`**. Left first, then right, then current

`[1, 2, 6, 7, 4]`

> **`inorder()`**. Left first, then current, then right

`[1, 2, 4, 6, 7]`

In [6]:
class BSTNode:
#Preorder traversal is a way to visit all the nodes in the tree. This produces a list
    #with the current tree values
    def preorder(self, visited):
        if self.val:
            visited.append(self.val)
        if self.left:
            self.left.preorder(visited)
        if self.right:
            self.right.preorder(visited)
        return visited
    
    #Postorder traversal also visits each node in a tree. The difference is that the current
    #node is visited afters its children
    def postorder(self, visited):
        if self.left:
            self.left.postorder(visited)
        if self.right:
            self.right.postorder(visited)
        if self.val:
            visited.append(self.val)
        return visited

    #Returns a list of values of the tree, with the current node betweeen its children
    def inorder(self, visited):
        if self.left:
            self.left.inorder(visited)
        if self.val:
            visited.append(self.val)
        if self.right:
            self.right.inorder(visited)
        return visited
    

## Node **`exists()`**

This method uses a similar structure to traverse, but instead of looking for the value in both of the branches (left and right if they are not None), it uses a comparison to decide which branch to iterate on

In [7]:
class BSTNode:
    def exists(self, val):
        #base case
        if self.val == val:
            return True
        if val < self.val and self.left:
            return self.left.exists(val)
        if val > self.val and self.right:
            return self.right.exists(val)
        return False

## **`height()`**

When the code arrives to the very last node (leaf node), and it returns 0, it goes up and, in the very bottom of the three, this happens:
max(0,0) + 1. 

Then, each level adds +1 for itself on top of the maximum of its children's heights

In [ ]:
def height(self):
        if self.val == None:
            return 0

        left_heigh = 0
        if self.left:
            left_heigh += self.left.height()

        right_heigh = 0
        if self.right:
            right_heigh += self.right.height()
            
        return max(right_heigh, left_heigh) + 1

# Final version of the class BSTNode

In [ ]:
class BSTNode:
    def __init__(self, val=None):
        self.left = None
        self.right = None
        self.val = val

    def insert(self, val): #O(log(n))

        if self.val is None:
            self.val = val
            return
        
        if self.val == val:
             return

        #if the left and right does not exits 
        if val < self.val and self.left is None:
            self.left = BSTNode(val)

        elif val > self.val and self.right is None:
            self.right= BSTNode(val)

        #if the left and right exists, call recursively the method over the child
        elif val < self.val and self.left:
            self.left.insert(val)

        elif val > self.val and self.right:
            self.right.insert(val)

    def delete(self, val): #O(log(n))
        #Empty tree or a leaf node where deletion has ocurred
        if self.val is None:
            return None
        
        if val < self.val:
            if self.left:
                self.left = self.left.delete(val)
                return self

        if val > self.val:
            if self.right is not None:
                self.right = self.right.delete(val)
                return self

        #value to delete found
        if val == self.val:
            #replacing the node to delete with its child if there is only one of them
            if not self.right:
                return self.left
            if not self.left:
                return self.right

            #Deleting a node with two children
            if self.right and self.left:
                succesor = self.right
                #This ensures that the succesor will be the smallest value from the right
                #so the rule of l < current node < r is actually followed
                while succesor.left is not None:
                    succesor = succesor.left
                self.val = succesor.val
                #This deletes the values on the right recursively, so the whole BST is 
                #restructured around the deleted val
                self.right = self.right.delete(succesor.val)
                return self

    #Preorder traversal is a way to visit all the nodes in the tree. This produces a list
    #with the current tree values
    def preorder(self, visited):
        if self.val is not None:
            visited.append(self.val)
        if self.left is not None:
            self.left.preorder(visited)
        if self.right is not None:
            self.right.preorder(visited)
        return visited
    
    #Postorder traversal also visits each node in a tree. The difference is that the current
    #node is visited afters its children
    def postorder(self, visited):
        if self.left:
            self.left.postorder(visited)
        if self.right:
            self.right.postorder(visited)
        if self.val:
            visited.append(self.val)
        return visited

    #Returns a list of values of the tree, with the current node betweeen its children
    def inorder(self, visited):
        if self.left:
            self.left.inorder(visited)
        if self.val:
            visited.append(self.val)
        if self.right:
            self.right.inorder(visited)
        return visited

    def exists(self, val):
        #base case
        if self.val == val:
            return True
        if val < self.val and self.left:
            return self.left.exists(val)
        if val > self.val and self.right:
            return self.right.exists(val)
        return False

    def height(self):
        if self.val == None:
            return 0

        left_heigh = 0
        if self.left:
            left_heigh += self.left.height()

        right_heigh = 0
        if self.right:
            right_heigh += self.right.height()
            
        return max(right_heigh, left_heigh) + 1
    
    #These are some of the simpler BST algorithms
    def get_min(self):
        if self.left is None:
            return self.val
        elif self.left is not None:
            return self.left.get_min()

    def get_max(self):
        if self.right is None:
            return self.val
        elif self.right is not None:
            return self.right.get_max()